<a href="https://colab.research.google.com/github/valentinaflores/SDistribuidos/blob/main/producto_mv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mpi4py

In [2]:
from mpi4py import MPI
import numpy as np

In [3]:
# Inicializar el entorno MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [11]:
# Dimensiones de la matriz y el vector
matrix_rows = 3
matrix_cols = 3
vector_size = 3


In [12]:
# Generar matriz y vector aleatorios en el proceso raíz
if rank == 0:
    matrix = np.random.rand(matrix_rows, matrix_cols)
    vector = np.random.rand(vector_size)
else:
    matrix = None
    vector = None

In [13]:
# Distribuir la matriz y el vector a todos los procesos
matrix = comm.bcast(matrix, root=0)
vector = comm.bcast(vector, root=0)

In [14]:
# Calcular la porción de la matriz y el vector para cada proceso
local_matrix = np.zeros((matrix_rows // size, matrix_cols))
local_vector = np.zeros(vector_size)
comm.Scatter(matrix, local_matrix, root=0)
comm.Scatter(vector, local_vector, root=0)

In [15]:
# Realizar la multiplicación local del vector por la matriz
local_result = np.dot(local_matrix, local_vector)

In [16]:
# Realizar la reducción de suma para obtener el resultado final
result = np.zeros(matrix_rows)
comm.Allreduce(local_result, result, op=MPI.SUM)

In [17]:
# Imprimir el resultado en el proceso raíz
if rank == 0:
    print("Resultado de la multiplicación:")
    print(result)

Resultado de la multiplicación:
[0.467388   0.81716015 1.0757677 ]
